In [7]:
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [2]:
from sklearn.model_selection import train_test_split # split the dataset
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler 

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR 

In [4]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score

In [ ]:
df = pd.read_csv(r'../input/temperature-forecast-project-using-ml/temp.csv')
df.shape
pd.set_option('display.max_columns', 30)

In [ ]:
data = df.copy()

In [ ]:
data.info()

In [ ]:
data.isna().sum()

In [ ]:
data['Date'][0].split('-')

['30', '06', '2013']


In [5]:
list_of_year = []
list_of_month = []
list_of_day = []


for i in data['Date']:
    try :
        split_obj = i.split('-')
        list_of_year.append(split_obj[2])
        list_of_month.append(split_obj[1])    
        list_of_day.append(split_obj[0]) 
    except AttributeError:
        list_of_year.append(np.nan)
        list_of_month.append(np.nan)
        list_of_day.append(np.nan)

In [ ]:
data['year'] = list_of_year
data['month'] = list_of_month
data['day'] = list_of_day

In [ ]:
data['year'] = pd.to_numeric(data['year']) 
data['month'] = pd.to_numeric(data['month']) 
data['day'] = pd.to_numeric(data['day'])

In [ ]:
data.drop('Date', axis=1, inplace=True)

In [ ]:
data.head()

In [8]:
def plot_line(para):
    df = data.groupby(by=para).mean().reset_index()
    
    fig = px.line(df, x=df[para], y=['Present_Tmax', 'Present_Tmin'])
    fig.update_layout(template='plotly_dark')
    return fig.show()

In [ ]:
plot_line('day')

In [ ]:
data.columns

In [ ]:
data.head()


In [ ]:
data.drop('station', axis=1, inplace=True)

In [ ]:
def treat_nan(df):
    for i in df.columns:
        df[i].fillna(df[i].mean(), inplace=True)

In [ ]:
treat_nan(data)

In [ ]:
data.head()

In [ ]:
X = data.drop(['Next_Tmax', 'Next_Tmin'], axis=1)
y1 = data['Next_Tmax']
y2 = data['Next_Tmin']

In [ ]:
X.head()

In [ ]:
def normalizer(x_train, x_test):
    scaler = Normalizer()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)
    return x_train, x_test

In [ ]:
def minmax(x_train, x_test):
    scaler = MinMaxScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)
    return x_train, x_test

In [12]:
def stdscaler(x_train, x_test):
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)
    return x_train, x_test

In [ ]:
def best_model(X, y, scaler, algo):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True)

    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)

    X_train, X_test = scaler(X_train, X_test)

    model = algo()

    model.fit(X_train, y_train)

    predictions = model.predict(X_test)

    mae = mean_absolute_error(predictions, y_test)
    mse = mean_squared_error(predictions, y_test)

    print(f'The MAE is {mean_absolute_error(predictions, y_test)}')
    print(f'The MSE is {mean_squared_error(predictions, y_test)}')

    fig = px.scatter(x=predictions, y=y_test, template='plotly_dark', title=f'Actual Productivity vs Predictions')
    fig.update_traces(hovertemplate='Predicted Value : %{x} <br> Actual Value: %{y}')
    fig.update_layout(hoverlabel=dict(
        font_size = 20,
        bgcolor = 'white', 
        font_family = 'Helvetica'
    ))
    fig.update_xaxes(title='Predicted Values', showgrid=False)
    fig.update_yaxes(title='Actual Values', showgrid=False)

    return predictions, y_test, mse, mae, fig.show()

In [ ]:
list_of_target = [y1, y2]

list_of_algos = [LinearRegression, KNeighborsRegressor, DecisionTreeRegressor, RandomForestRegressor, SVR]

list_of_MAE = []
list_of_MSE = []

for i in list_of_algos:
    print(f"{i}")
    pred, y_test, mse, mae, plot = best_model(X, y1, normalizer, i)
    list_of_MSE.append(mse)
    list_of_MAE.append(mae)

In [ ]:
msemae_y1 = pd.DataFrame()
msemae_y1['Algos'] =  ['LinearRegression', 'KNeighborsRegressor', 'DecisionTreeRegressor', 'RandomForestRegressor', 'SVR']
msemae_y1['MSE_normalizer'] = list_of_MSE
msemae_y1['MAE_normalizer'] = list_of_MAE 

In [ ]:
msemae_y1.head()

In [ ]:
list_of_MAE1 = []
list_of_MSE1 = []

for i in list_of_algos:
    print(f"{i}")
    pred, y_test, mse, mae, plot = best_model(X, y1, stdscaler, i)
    list_of_MSE1.append(mse)
    list_of_MAE1.append(mae)

In [ ]:
msemae_y1['MSE_stdscaler'] = list_of_MSE1
msemae_y1['MAE_stdscaler'] = list_of_MAE1 

In [ ]:
list_of_MAE2 = []
list_of_MSE2 = []

for i in list_of_algos:
    print(f"{i}")
    pred, y_test, mse, mae, plot = best_model(X, y1, minmax, i)
    list_of_MSE2.append(mse)
    list_of_MAE2.append(mae)

In [ ]:
msemae_y1['MSE_minmax'] = list_of_MSE2
msemae_y1['MAE_minmax'] = list_of_MAE2 

In [ ]:
msemae_y1.head()

In [ ]:
list_of_algos = [LinearRegression, KNeighborsRegressor, DecisionTreeRegressor, RandomForestRegressor, SVR]

list_of_MAE3 = []
list_of_MSE3 = []

for i in list_of_algos:
    print(f"{i}")
    pred, y_test, mse, mae, plot = best_model(X, y2, normalizer, i)
    list_of_MSE3.append(mse)
    list_of_MAE3.append(mae)

In [ ]:
msemae_y2 = pd.DataFrame()
msemae_y2['Algos'] =  ['LinearRegression', 'KNeighborsRegressor', 'DecisionTreeRegressor', 'RandomForestRegressor', 'SVR']
msemae_y2['MSE_normalizer'] = list_of_MSE3
msemae_y2['MAE_normalizer'] = list_of_MAE3

In [ ]:
list_of_MAE4 = []
list_of_MSE4 = []

for i in list_of_algos:
    print(f"{i}")
    pred, y_test, mse, mae, plot = best_model(X, y2, stdscaler, i)
    list_of_MSE4.append(mse)
    list_of_MAE4.append(mae)

In [ ]:
msemae_y2['MSE_stdscaler'] = list_of_MSE4
msemae_y2['MAE_stdscaler'] = list_of_MAE4

In [ ]:
list_of_MAE5 = []
list_of_MSE5 = []

for i in list_of_algos:
    print(f"{i}")
    pred, y_test, mse, mae, plot = best_model(X, y2, minmax, i)
    list_of_MSE5.append(mse)
    list_of_MAE5.append(mae)

In [ ]:
msemae_y2['MSE_minmax'] = list_of_MSE5
msemae_y2['MAE_minmax'] = list_of_MAE5

In [ ]:
msemae_y2.head()

In [ ]:
fig = px.bar(msemae_y2,  x='Algos', y=['MSE_normalizer', 'MAE_normalizer', 'MSE_stdscaler', 'MAE_stdscaler', 'MSE_minmax', 'MAE_minmax'], barmode='group')
fig.update_layout(title='Representation of MAE and MSE values of different Algorithms on the Next Min Temperature', template='plotly_dark', hoverlabel=dict(
    font_size=20,
    font_family='Arial'
))
fig.update_traces(hovertemplate='%{x} : %{y}')
fig.show()